In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mlflow.pipelines

help(mlflow.pipelines)

Help on package mlflow.pipelines in mlflow:

NAME
    mlflow.pipelines - Fill out help string later

PACKAGE CONTENTS
    cli
    evaluate_step
    notebook
    split_step
    train_step
    transform_step

FUNCTIONS
    clean()
        Clean
    
    evaluate()
        Evaluate a model (explanations included)
    
    ingest()
        Ingest data
    
    inspect()
        Inspect specific steps or full pipeline DAG
    
    split()
        Split data
    
    train()
        Train a model
    
    transform()
        Transform features

FILE
    /Users/corey.zumar/mlflow/mlflow/pipelines/__init__.py




In [8]:
mlflow.pipelines.clean()

rm -rf split_train.parquet split_test.parquet split_summary.html transform_transformer.pkl transform_train_transformed.parquet train_pipeline.pkl train_run_id evaluate_worst_training_examples.parquet evaluate_metrics.json evaluate_explanations.html


In [3]:
mlflow.pipelines.ingest()

In [4]:
mlflow.pipelines.split()

make: Nothing to be done for `split'.
== Showing summary of input data ==

Split data into train/test sets
== Summary of train data ==

        symboling  normalized-losses  wheel-base      length       width  \
count  159.000000         131.000000  159.000000  159.000000  159.000000   
mean     0.855346         121.251908   98.500629  173.499371   65.708176   
std      1.210902          35.057391    6.218414   12.464682    2.044256   
min     -2.000000          65.000000   86.600000  141.100000   60.300000   
25%      0.000000          94.000000   94.500000  166.300000   64.000000   
50%      1.000000         115.000000   96.500000  172.400000   65.400000   
75%      2.000000         148.000000  100.400000  177.800000   66.500000   
max      3.000000         256.000000  120.900000  208.100000   72.000000   

           height  curb-weight  engine-size        bore      stroke  \
count  159.000000   159.000000   159.000000  156.000000  156.000000   
mean    53.736478  2520.716981   124.

In [5]:
mlflow.pipelines.transform()

python -c "from mlflow.pipelines.transform_step import run_transform_step; run_transform_step(train_data_path='split_train.parquet', transformer_config_path='steps/transformer_config.yaml', transformer_output_path='transform_transformer.pkl', transformed_data_output_path='transform_train_transformed.parquet')"
== Summary of transformed features ==

               0           1           2           3           4           5   \
count  159.000000  159.000000  159.000000  159.000000  159.000000  159.000000   
mean     0.018868    0.025157    0.037736    0.018868    0.044025    0.075472   
std      0.136488    0.157097    0.191159    0.136488    0.205799    0.264986   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
75%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
m

In [6]:
mlflow.pipelines.train()

python -c "from mlflow.pipelines.train_step import run_train_step; run_train_step(transformed_train_data_path='transform_train_transformed.parquet', train_config_path='steps/train_config.yaml', transformer_path='transform_transformer.pkl', tracking_uri='file:/tmp/mlruns', pipeline_output_path='train_pipeline.pkl', run_id_output_path='train_run_id')"


/Users/corey.zumar/mlflow/mlflow/pipelines/train_step.py:25: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  module_name, method_name = yaml.load(open(train_config_path, "r")).get("train_method").rsplit('.', 1)
2022/04/14 13:47:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/04/14 13:47:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2022/04/14 13:47:59 INFO mlflow.pyspark.ml: No SparkSession detected. Autologging will log pyspark.ml models contained in the default allowlist. To specify a custom allowlist, initialize a SparkSession prior to calling mlflow.pyspark.ml.autolog() and specify the path to your allowlist file via the spark.mlflow.pysparkml.autolog.logModelAllowlistFile conf.
2022/04/14 13:47:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.


== Trained a model at train_pipeline.pkl ==



In [7]:
mlflow.pipelines.evaluate()

python -c "from mlflow.pipelines.evaluate_step import run_evaluate_step; run_evaluate_step(pipeline_path='train_pipeline.pkl', tracking_uri='file:/tmp/mlruns', run_id_path='train_run_id', train_data_path='split_train.parquet', test_data_path='split_test.parquet', explanations_output_path='evaluate_explanations.html', metrics_output_path='evaluate_metrics.json', worst_train_examples_output_path='evaluate_worst_training_examples.parquet')"


100%|██████████| 10/10 [00:02<00:00,  3.95it/s]


== Created the model card ==

== Produced evaluation metrics ==

